## Automated ML

Import dependencies. 

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

## Dataset

### Overview

The Iris dataset was used in R.A. Fisher's classic 1936 paper, The Use of Multiple Measurements in Taxonomic Problems, and can also be found on the UCI Machine Learning Repository.

It includes three iris species with 50 samples each as well as some properties about each flower. One flower species is linearly separable from the other two, but the other two are not linearly separable from each other.

The columns in this dataset are:

Id
SepalLengthCm
SepalWidthCm
PetalLengthCm
PetalWidthCm
Species

## Workspace setup

In [ ]:
ws = Workspace.from_config()

experiment_name = 'automl-experiment'

experiment = Experiment(ws, experiment_name)

In [ ]:
dataset = Dataset.get_by_name(ws, 'iris-dataset-kaggle')

In [ ]:
dataset.take(5).to_pandas_dataframe()

In [ ]:
data_train, data_test = dataset.random_split(0.9)

## Config Compute Cluster

In [ ]:
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## AutoML Configuration

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=data_train,
                             label_column_name="Species", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings
                            )

AutoML typically performs cross validation, data balance check, cardinality check in prior to machine learning process with a variety of algorithms. 

In [ ]:
remote_run = experiment.submit(automl_config, show_output = True)

## Run Details

In [ ]:
remote_run

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion()

## Best Model 

In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [ ]:
best_run

In [ ]:
fitted_model

In [ ]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(fitted_model._final_estimator)
print(best_run.get_tags())

In [ ]:
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

model_name = best_run.properties['model_name']
model_name

In [ ]:
env = best_run.get_environment()

script_file = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

## Model Deployment

In [ ]:
#Register the fitted model
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML model')

In [ ]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

aci_service_name = 'automl-iris-classifier'
print(aci_service_name)

In [ ]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

## Testing using 2 different methods

In [ ]:
%run endpoint.py

In [ ]:
#data_test = data_test.to_pandas_dataframe().dropna()
data_sample = data_test.sample(3)
y_true = data_sample.pop('Species')
sample_json = json.dumps({'data':data_sample.to_dict(orient='records')})
print(sample_json)

In [ ]:
output = service.run(sample_json)
print('Prediction: ', output)
print('True Values: ', y_true.values)

In [ ]:
service.get_logs()

In [ ]:
service.delete()